In [2]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df_cendus_poi = pd.read_pickle('../clean_pickles/target_cendus_poi.pk')
df_cendus_poi_sub = pd.read_pickle('../clean_pickles/target_cendus_poi_sub.pk')
df_dianping_poi = pd.read_pickle('../clean_pickles/target_dianping_poi.pk')
df_dianping_poi_cat = pd.read_pickle('../clean_pickles/target_dianping_poi_cat.pk')
df_mobike = pd.read_pickle('../clean_pickles/target_mobike.pk')
df_mobike_byhours = pd.read_pickle('../clean_pickles/target_mobike_byhours.pk')
df_re = pd.read_pickle('../clean_pickles/target_real_estate.pk')
df_taxi = pd.read_pickle('../clean_pickles/target_taxi_speed.pk')
df_weibo = pd.read_pickle('../clean_pickles/target_weibo.pk')

In [4]:
df_pred = df_cendus_poi.iloc[:,9:len(df_cendus_poi.columns)]
df_pred = df_pred.join(df_cendus_poi_sub.iloc[:,9:len(df_cendus_poi_sub.columns)], rsuffix='_sub')
df_pred = df_pred.join(df_dianping_poi.iloc[:,9:len(df_dianping_poi.columns)])
df_pred = df_pred.join(df_dianping_poi_cat.iloc[:,9:len(df_dianping_poi_cat.columns)])
df_pred = df_pred.join(df_mobike.iloc[:,9:len(df_mobike.columns)])
df_pred = df_pred.join(df_mobike_byhours.iloc[:,9:len(df_mobike_byhours.columns)])
df_pred = df_pred.join(df_re.iloc[:,9:len(df_re.columns)])
df_pred = df_pred.join(df_taxi.iloc[:,9:len(df_taxi.columns)])
df_pred = df_pred.join(df_weibo.iloc[:,9:len(df_weibo.columns)])

### Number of criterias

In [5]:
len(df_pred.columns)

149

In [6]:
def cleanCriteriasNoiseSmell(df):
    droppingColumns = []
    nb0 = df.iloc[:,len(df.columns)-1].value_counts()[0]
    nb1 = df.iloc[:,len(df.columns)-1].value_counts()[1]
    ratio = nb1 / nb0
    counter = 0
    for i in range(0,len(df.columns)-1):
        # 10 NA max to remove a maximum of 3 % of the points
        if len(df[df.iloc[:,i].isna()]) > 10:
            nbi0 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[0]
            try:
                nbi1 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[1]
            except KeyError:
                nbi1 = 0
            #if the proportion of the class is kept then we keep the criterion
            if (nbi1/nbi0) > ratio:
                droppingColumns.append(df.iloc[:,i].name)
                counter +=1
                print(df.iloc[:,i].name,  len(df[df.iloc[:,i].isna()]))
    print(counter)
    return df.drop(columns=droppingColumns)
    

In [7]:
def cleanCriteriasClean(df):
    droppingColumns = []
    nb1 = df.iloc[:,len(df.columns)-1].value_counts()[1]
    nb2 = df.iloc[:,len(df.columns)-1].value_counts()[2]
    nb3 = df.iloc[:,len(df.columns)-1].value_counts()[3]
    nb4 = df.iloc[:,len(df.columns)-1].value_counts()[4]
    ratio = (nb1+nb4) / (nb2+nb3)
    counter = 0
    for i in range(0,len(df.columns)-1):
        # 10 NA max to remove a maximum of 3 % of the points
        if len(df[df.iloc[:,i].isna()]) > 10:
            try:
                nbi1 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[1]
            except KeyError:
                nbi1 = 0
            try:
                nbi2 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[2]
            except KeyError:
                nbi2 = 0
            try:
                nbi3 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[3]
            except KeyError:
                nbi3 = 0
            try:
                nbi4 = df[df.iloc[:,i].isna()].iloc[:,len(df.columns)-1].value_counts()[4]
            except KeyError:
                nbi4 = 0
            #if the proportion of the class is kept then we keep the criterion
            if ( (nbi1+nbi4)/(nbi2+nbi3) ) > ratio:
                droppingColumns.append(df.iloc[:,i].name)
                counter +=1
                print(df.iloc[:,i].name,  len(df[df.iloc[:,i].isna()]))
    print(counter)
    return df.drop(columns=droppingColumns)

We display the criteria that have been removed

In [8]:
df_pred_clean = df_pred.join(df_cendus_poi['clean'])
df_pred_clean = df_pred_clean.dropna(subset=['clean'])
df_pred_clean = cleanCriteriasClean(df_pred_clean)
df_pred_clean = df_pred_clean.dropna()
df_pred_clean = df_pred_clean.sample(n=len(df_pred_clean), random_state=3)

av_price_dianping 18
av_decoration_score_dianping 19
av_product_score_dianping 19
av_service_score_dianping 19
av_mobike_duration 47
av_mobike_distance 47
av_mobike_speed 47
av_friends_count_weibo 68
av_followers_count_weibo 68
av_status_count_weibo 68
av_total_checkin_count_weibo 68
av_tip_count_weibo 68
12


In [9]:
df_pred_noise = df_pred.join(df_cendus_poi['noise'])
df_pred_noise = df_pred_noise.dropna(subset=['noise'])
df_pred_noise = cleanCriteriasNoiseSmell(df_pred_noise)
df_pred_noise = df_pred_noise.dropna()
df_pred_noise = df_pred_noise.sample(n=len(df_pred_noise), random_state=3)

av_price_dianping 16
av_decoration_score_dianping 18
av_product_score_dianping 18
av_service_score_dianping 18
av_mobike_duration 39
av_mobike_distance 39
av_mobike_speed 39
taxi_speed_mean 12
av_friends_count_weibo 60
av_followers_count_weibo 60
av_status_count_weibo 60
av_total_checkin_count_weibo 60
av_tip_count_weibo 60
13


In [10]:
df_pred_smell = df_pred.join(df_cendus_poi['smell'])
df_pred_smell = df_pred_smell.dropna(subset=['smell'])
df_pred_smell = cleanCriteriasNoiseSmell(df_pred_smell)
df_pred_smell = df_pred_smell.dropna()
df_pred_smell = df_pred_smell.sample(n=len(df_pred_smell), random_state=3)

av_price_dianping 18
av_decoration_score_dianping 18
av_product_score_dianping 18
av_service_score_dianping 18
av_mobike_duration 43
av_mobike_distance 43
av_mobike_speed 43
av_onesquaremeter_re 16
av_bedroom_re 16
taxi_speed_mean 12
av_friends_count_weibo 63
av_followers_count_weibo 63
av_status_count_weibo 63
av_total_checkin_count_weibo 63
av_tip_count_weibo 63
15


# Prediction

## Noise

In [11]:
X = df_pred_noise.iloc[:,0:len(df_pred_noise.columns)-1].copy()
#X = df_pred_noise[['nb_shopping_cendus']].copy()
Y = df_pred_noise['noise'].copy()

It's important to scale value for QDA

In [12]:
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

In [13]:
Y.value_counts()

0.0    344
1.0     87
Name: noise, dtype: int64

Proportion of the class 0:

In [14]:
prop = Y.value_counts()
prop[0.0] / (prop[0.0] + prop[1.0]) * 100

79.81438515081206

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA

In [16]:
clf = QDA()
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.8027800053461641

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=True)
clf = QDA()
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.8251748251748252

In [18]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred)

0.07407407407407408

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[117,   3],
       [ 22,   1]], dtype=int64)

I would suggest here to do a PCA (Principal component analysis) to remove the correlation between my criteria.

## Clean

In [37]:
X = df_pred_clean.iloc[:,0:len(df_pred_clean.columns)-1].copy()
#X = df_cendus_noise[['nb_shopping_cendus']].copy()
Y = df_pred_clean['clean'].copy()
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

In [38]:
Y.value_counts()

3    218
2    182
4     17
1     15
Name: clean, dtype: int64

Proportion of the class 2 & 3:

In [39]:
prop = Y.value_counts()
(prop[2.0]+prop[3.0]) / (prop[1.0] + prop[2.0] + prop[3.0] + prop[4.0]) * 100

92.5925925925926

In [40]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
clf = QDA()
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.4746591820368885

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=True)
clf = QDA()
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.4755244755244755

In [42]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='micro')

0.4755244755244755

In [43]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='weighted')

0.33803617969743865

In [44]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[ 0,  0,  5,  0],
       [ 0,  2, 60,  0],
       [ 1,  3, 66,  0],
       [ 0,  0,  6,  0]], dtype=int64)

In [45]:
Y_val.value_counts()

3    70
2    62
4     6
1     5
Name: clean, dtype: int64

In [46]:
pd.Series(Y_pred).value_counts()

3    137
2      5
1      1
dtype: int64

## Smell

In [47]:
X = df_pred_smell.iloc[:,0:len(df_pred_smell.columns)-1].copy()
#X = df_cendus_noise[['nb_shopping_cendus']].copy()
Y = df_pred_smell['smell'].copy()
from sklearn.preprocessing import MinMaxScaler
X = MinMaxScaler().fit_transform(X)

In [48]:
Y.value_counts()

0.0    367
1.0     16
Name: smell, dtype: int64

Proportion of the class 0:

In [49]:
prop = Y.value_counts()
prop[0.0] / (prop[0.0] + prop[1.0]) * 100

95.822454308094

In [50]:
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
clf = QDA()
scores = cross_val_score(clf, X, Y, cv=5)
np.mean(scores)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.9477785372522215

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.33, shuffle=True)
clf = QDA()
clf = clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_val)
np.mean(Y_pred == Y_val)

C:\Users\Roger\Anaconda3\lib\site-packages\sklearn\discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


0.9291338582677166

In [52]:
from sklearn.metrics import f1_score
f1_score(Y_val, Y_pred, average='binary')

0.0

In [53]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_val, Y_pred)

array([[118,   2],
       [  7,   0]], dtype=int64)

In [54]:
Y_val.value_counts()

0.0    120
1.0      7
Name: smell, dtype: int64

In [55]:
pd.Series(Y_pred).value_counts()

0.0    125
1.0      2
dtype: int64